In [ ]:
# Install packages (execute only once if you don't have them)
# Execute the following in the terminal: pip install package_name
# OR directly from Jupyter notebook: !pip install package_name

In [1]:
# Load packages
import copy
from os.path import join, expanduser
import pandas, numpy, seaborn, matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
import rsatoolbox.io.meadows
from rsatoolbox.rdm.transform import rank_transform
from rsatoolbox.rdm.combine import from_partials, rescale
from rsatoolbox.vis.rdm_plot import show_rdm

In [2]:
# Define rough categorical order based on animacy (subjective)
animacy_order = [
    'LaughingAdultFemale',
    'LaughingAdultMale',
    'LaughingChild',
    'LaughingGroup',
    'CryingAdultFemale',
    'CryingAdultMale',
    'CryingChild',
    'ChildrenPlaying',
    'CocktailParty',
    'CrowdCheering',
    'CrowdApplause',
    'CoughingAdultFemale',
    'CoughingAdultMale',
    'Sneezing',
    'Snoring',
    'ThroatClearing',
    'Footsteps',
    'CatHissing',
    'CatMeowing',
    'CatPurring',
    'BirdSong',
    'BirdSquawk',
    'BirdsFlyingOff',
    'DogBarking',
    'DogGrowling',
    'DogHowling',
    'HorseGalloping',
    'HorseNeighing',
    'HorseSnorting',
    'CowMoo',
    'InsectBuzzing',
    'InsectsStridulation',
    'CarHorn',
    'CarSkidding',
    'CarStarting',
    'TrainBreaks',
    'TrainSignal',
    'TrainWagons',
    'Helicopter',
    'Propeller',
    'JetPassing',
    'DrillingPneumatic',
    'Drilling',
    'SawingManual',
    'Sawing',
    'Hammer',
    'Fire',
    'Rain',
    'River',
    'SeaWaves',
    'Thunder',
    'Waterfall',
    'Wind'
]

In [3]:
# Volume check settings
T_VOLCHECK = 3
T_MULTIARRANGE = 6
N_TRIALS_VOLCHECK = 6
UNUSED_COLS = ['task', 'stim1_id', 'stim2_id', 'stim3_id',
    'stim1_name', 'stim2_name', 'stim3_name', 'start', 'resp']
PASSED_FAILED = {True: 'passed', False: 'FAILED'}

In [4]:
# Folder with the data (change to your location)
root_dir = expanduser('/Users/mkachlicka/Documents/EnviSounds_Meadows/pilotdata/')

# Include all the versions of the task as downloaded from Meadows (check the filenames, e.g., v_v17)
versions = [17, 18] 
versions_df = []
for version in versions:
    # '_metadata.csv' file contains the list of all participants and their status
    # Relevant data: name (Meadows ID), status (finished, timedout), dur_mins (time spent on the task),
    # started (time start), progressed (time end), PROLIFIC_PID (Prolific ID)
    fname_meta = f'Meadows_Sorting_Task_v_v{version}_metadata.csv'
    version_df = pandas.read_csv(join(root_dir, fname_meta), index_col='name')
    version_df['version'] = version
    versions_df.append(version_df)
df_meta = pandas.concat(versions_df)
subjects = df_meta[df_meta.tasks_finished>6] # df_meta.status=='finished'
n_stim = len(animacy_order)
count = numpy.zeros([n_stim, n_stim], dtype=numpy.int)
n = 0
rdm_list = []
for name, subject in subjects.iterrows():

    ## Volume check '_annotations.csv' files: load data
    # Relevant data: participation (Meadows ID), stim1_name (sound sequence, last digit represents correct answer),
    # label (actual answer); more than 6 rows per participant mean that they repeated the volume check
    fname_annot = f'Meadows_Sorting_Task_v_v{subject.version}_{subject.name}_4_annotations.csv'
    df_annot = pandas.read_csv(join(root_dir, fname_annot))
    df_annot['expected'] = df_annot.stim1_name.str.split('_').str[2].astype(int)
    df_annot = df_annot.drop(UNUSED_COLS, axis=1)
    nAttempts = round(len(df_annot)/N_TRIALS_VOLCHECK)
    correct = df_annot.label == df_annot.expected
    passed = all(correct.values[-6:])
    # Prints status for all participants and how many attempts
    print(f'{subject.name}: {PASSED_FAILED[passed]} after {nAttempts} attempts')

    if not passed:
        continue
        
    ## Spatial arrangement data '_7_1D.mat' files: load data
    fname_mat = f'Meadows_Sorting_Task_v_v{subject.version}_{subject.name}_7_1D.mat'
    mat_fpath = join(root_dir, fname_mat)
    rdm = rsatoolbox.io.meadows.load_rdms(mat_fpath, sort=False)
    rdm.pattern_descriptors['name'] = [s.split('_')[0] for s in rdm.pattern_descriptors['conds']]
    rdm_list.append(rdm)

    subject_stim_names = rdm.pattern_descriptors['name']
    sparse_idx = [animacy_order.index(s) for s in subject_stim_names]
    count[numpy.ix_(sparse_idx, sparse_idx)] += 1
    n += 1

# Create weighted RDM from individual spatial arrangements (dissimilarity measure = Euclidean dissimilarities)
partials = from_partials(rdm_list, descriptor='name')
rescaledPartials = rescale(partials, method='evidence')
meanRDM = rescaledPartials.mean(weights='rescalingWeights')
animacy_idx = [animacy_order.index(s) for s in meanRDM.pattern_descriptors['name']]
new_order = numpy.argsort(animacy_idx)
meanRDM.reorder(new_order)

# Save RDM as png file
fig, _, _ = show_rdm(rank_transform(meanRDM), pattern_descriptor='name', figsize=(12,12), show_colorbar='panel')
fig.savefig(f'rdm_n{n}.png', dpi=200)
plt.close(fig)

<ipython-input-4-4fdf4621711a>:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  count = numpy.zeros([n_stim, n_stim], dtype=numpy.int)


natural-macaw: passed after 2 attempts
massive-boa: passed after 1 attempts
informed-condor: passed after 1 attempts
artistic-pelican: passed after 1 attempts
regular-kit: passed after 2 attempts
social-buffalo: passed after 1 attempts
humane-puma: passed after 1 attempts
growing-stud: passed after 2 attempts
equal-moose: passed after 1 attempts
selected-husky: FAILED after 2 attempts
engaged-chigger: passed after 1 attempts
summary-seal: passed after 1 attempts
awaited-tomcat: passed after 1 attempts
helping-parrot: passed after 1 attempts
beloved-jackal: passed after 1 attempts
welcome-toucan: passed after 1 attempts
internal-panther: passed after 1 attempts
leading-mastodon: FAILED after 2 attempts
infinite-falcon: passed after 1 attempts
precious-chicken: passed after 1 attempts
tops-meerkat: passed after 1 attempts
noble-ape: passed after 1 attempts
daring-prawn: passed after 1 attempts
evolved-javelin: FAILED after 2 attempts
able-vervet: FAILED after 2 attempts
sacred-hamster: F

In [5]:
# Here you can have a look at the individual spatial arrangements data
print(rdm_list)

[rsatoolbox.rdm.RDMs(
dissimilarity_measure = 
euclidean
dissimilarities = 
[[0.05522687 0.09295991 0.06161647 0.08266499 0.08485304 0.05543333
  0.09031186 0.04496834 0.08922276 0.01663698 0.05006872 0.06093565
  0.08703628 0.04475149 0.04474295 0.04008637 0.0818881  0.08184137
  0.09068466 0.09276038 0.05774317 0.09528177 0.02849679 0.02053478
  0.06951663 0.0701897  0.09643969 0.05651363 0.07384135 0.07084148
  0.07068209 0.05042966 0.06587587 0.06540009 0.07239043 0.03165332
  0.05797499 0.10097188 0.05927507 0.10536582 0.08839997 0.06957322
  0.11894566 0.03584446 0.1063869  0.05136016 0.05007467 0.06551975
  0.07533351 0.05320919 0.08257388 0.04271172 0.08540238 0.0777134
  0.0889612  0.09770551 0.06265188 0.11027939 0.0493409  0.09792437
  0.04359808 0.05568145 0.03920608 0.10976003 0.06803644 0.06111037
  0.05250098 0.10569093 0.09488103 0.09790055 0.10325594 0.09286822
  0.05331148 0.11980287 0.06907013 0.10900677 0.07546081 0.07975419
  0.05059202 0.05724642 0.07246095 0.0574

In [6]:
# Space coverage check (counts participants who passed volume check, counts ratings per pair)
numpy.fill_diagonal(count, 0)
count_utv = squareform(count)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
cmap = copy.copy(matplotlib.cm.get_cmap('Reds'))
cmap.set_under(color='black')    
seaborn.heatmap(count, cmap=cmap, vmin=0.01, ax=axes[0], cbar_kws=dict(ticks=numpy.unique(count_utv)))
df_count = pandas.DataFrame(count_utv, columns=['count'])

# Prints coverage count per N participants
print(df_count['count'].value_counts().sort_index())
print(f'\n total participants passed: {n}')
seaborn.histplot(data=df_count, x='count', ax=axes[1], color='red')
axes[1].set_ylim(0, 800)
fig.suptitle(f'{n} participants')
fig.savefig(f'count_n{n}.png')
plt.close(fig)

1       3
2      15
3      39
4     117
5     174
6     231
7     240
8     209
9     164
10    103
11     49
12     25
13      8
15      1
Name: count, dtype: int64

 total participants passed: 51


In [7]:
# Save RDM (distance values) as csv file
rawRDM = meanRDM.get_matrices()
x = rawRDM[0, :, :] # Reducing dimensions of ndarray to 2D
numpy.savetxt("pilotrdm.csv", x, delimiter=",") # Now has commas as delimiters